In [1]:
import pandas as pd
import os
import json


Let's walk through the input folder and save every bill as its json contents.


In [7]:
bills = []
statesDict = {}
for f in os.walk(os.getcwd() + "\\input\\"):
    if "bill" in f[0]:
        stateAbb = f[0][f[0].index("input") + 6: f[0].index("input") + 8]
        for file in f[-1]:
            if ".json" in file:
                fileName = f[0] + "\\" + file
                with open(fileName, "r") as b:
                    s = json.loads(b.read())
                    bills.append(s)
                    statesDict[s['bill']['session']['state_id']] = stateAbb


`statesDict` contains the mapping of all internal state_ids to their abbreviation, for ease of reading. This will be used later when storing information about a bill, vote, or person for human readable formats.

In [8]:
statesDict

{6: 'CO'}

Let's take a look at what one of our bill files looks like


In [18]:
print(json.dumps(bills[0], indent=2))


{
  "bill": {
    "bill_id": 1299551,
    "change_hash": "3caacb77278d3e402301e8d93bd50962",
    "session_id": 1722,
    "session": {
      "session_id": 1722,
      "state_id": 6,
      "year_start": 2020,
      "year_end": 2020,
      "prefile": 0,
      "sine_die": 1,
      "prior": 1,
      "special": 0,
      "session_tag": "Regular Session",
      "session_title": "2020 Regular Session",
      "session_name": "2020 Regular Session"
    },
    "url": "https://legiscan.com/CO/bill/HB1001/2020",
    "state_link": "https://leg.colorado.gov/bills/hb20-1001",
    "completed": 1,
    "status": 4,
    "status_date": "2020-07-14",
    "progress": [
      {
        "date": "2020-01-08",
        "event": 1
      },
      {
        "date": "2020-01-08",
        "event": 9
      },
      {
        "date": "2020-02-12",
        "event": 10
      },
      {
        "date": "2020-02-12",
        "event": 9
      },
      {
        "date": "2020-03-02",
        "event": 10
      },
      {
      

Ok that's a lot of information, most of it useless. Let's take out what we care about.


In [19]:
statuses = [
    "N\\A",
    "Intro",
    "Engrossed",
    "Enrolled",
    "Passed",
    "Vetoed",
    "Failed",
    "Override",
    "Chaptered",
    "Refer",
    "Report Pass",
    "Report DNP",
    "Draft",
]
bill_id = bills[0]["bill"]["bill_id"]
status = statuses[bills[0]["bill"]["status"]]
bill_name = bills[0]["bill"]["title"]
bill_number = bills[0]["bill"]["bill_number"]
bill_desc = bills[0]["bill"]["description"]
text_link = bills[0]["bill"]["texts"][-1]["url"]
sponsors = []
for person in bills[0]["bill"]["sponsors"]:
    sponsors.append(person["people_id"])
roll_call_ids = []
for vote in bills[0]["bill"]["votes"]:
    roll_call_ids.append(vote["roll_call_id"])
example_bill = {
    "ID": bill_id,
    "Name": bill_name,
    "Number": bill_number,
    "Status": status,
    "Description": bill_desc,
    "Text": text_link,
    "Sponsors": f"{sponsors}",
    "Votes": f"{roll_call_ids}",
}
example_bill


{'ID': 1299551,
 'Name': 'Nicotine Product Regulation',
 'Number': 'HB1001',
 'Status': 'Passed',
 'Description': 'Concerning nicotine product regulations, and, in connection therewith, making an appropriation.',
 'Text': 'https://legiscan.com/CO/text/HB1001/id/2198623',
 'Sponsors': '[20136, 20135, 19048, 3323]',
 'Votes': '[921854, 925968, 925969, 925970, 925971, 925972, 939781, 944787, 944943, 946767, 947762, 960290, 962073, 963123, 963235, 963236, 965916, 965917]'}

Much better, let's do that to every single one of our bills and add them to a DataFrame so we can save it out to a database later.


In [20]:
statuses = [
    "N\\A",
    "Intro",
    "Engrossed",
    "Enrolled",
    "Passed",
    "Vetoed",
    "Failed",
    "Override",
    "Chaptered",
    "Refer",
    "Report Pass",
    "Report DNP",
    "Draft",
]
bill_list = []
for bill in bills:
    bill_id = bill["bill"]["bill_id"]
    status = statuses[bill["bill"]["status"]]
    bill_name = bill["bill"]["title"]
    bill_number = bill["bill"]["bill_number"]
    bill_desc = bill["bill"]["description"]
    text_link = (
        bill["bill"]["texts"][-1]["url"] if len(bill["bill"]["texts"]) != 0 else ""
    )
    sponsors = []
    for person in bill["bill"]["sponsors"]:
        sponsors.append(person["people_id"])
    roll_call_ids = []
    for vote in bill["bill"]["votes"]:
        roll_call_ids.append(vote["roll_call_id"])
    bill_list.append(
        (
            {
                "ID": bill_id,
                "Name": bill_name,
                "Number": bill_number,
                "Description": bill_desc,
                "Status": status,
                "Text": text_link,
                "Sponsors": f"{sponsors}",
                "Votes": f"{roll_call_ids}",
            }
        )
    )


In [21]:
bill_df = pd.DataFrame.from_records(bill_list)
bill_df.set_index("ID", inplace=True)
bill_df


,Name,Number,Description,Status,Text,Sponsors,Votes
ID,,,,,,,
1299551,Nicotine Product Regulation,HB1001,"Concerning nicotine product regulations, and, ...",Passed,https://legiscan.com/CO/text/HB1001/id/2198623,"[20136, 20135, 19048, 3323]","[921854, 925968, 925969, 925970, 925971, 92597..."
1299535,College Credit For Work Experience,HB1002,Concerning a statewide plan for awarding colle...,Passed,https://legiscan.com/CO/text/HB1002/id/2197314,"[18711, 20126, 16096, 20032]","[906953, 935368, 951329, 951748, 963052, 96333..."
1299644,Rural Jump-start Zone Act Modifications,HB1003,Concerning modifications to the rural jump-sta...,Passed,https://legiscan.com/CO/text/HB1003/id/2197322,"[19512, 20137, 16569, 10778]","[906233, 916445, 917515, 917516, 944823, 94539..."
1299620,Assistance Landowner Wildfire Mitigation,HB1004,Concerning wildfire mitigation assistance for ...,Intro,https://legiscan.com/CO/text/HB1004/id/2088138,"[20131, 21293, 10780]","[905713, 905714, 905990, 911697, 911698, 91169..."
1299507,Enhance Safe2tell,HB1005,Concerning enhancements to the safe2tell progr...,Failed,https://legiscan.com/CO/text/HB1005/id/2166121,"[18713, 16578, 11607, 16509]","[917642, 924391, 944786, 945802, 947189, 960714]"
...,...,...,...,...,...,...,...
1298845,Concerning the appointment of officers and emp...,SR001,Concerning the appointment of officers and emp...,Passed,https://legiscan.com/CO/text/SR001/id/2088763,[18729],[962487]
1326520,Missing Persons Day,SR002,"Concerning the designation of February 4 as ""M...",Passed,https://legiscan.com/CO/text/SR002/id/2189974,"[3343, 11607]",[914588]
1356060,Single Parents' Day,SR003,"Concerning recognition of ""Single Parents' Day"".",Intro,https://legiscan.com/CO/text/SR003/id/2162168,"[20028, 16096]",[]


Alright, now for the hard (maybe?) part. We need to take votes for each bill and pull detailed information from the voting data provided by LegiScan.

First let's walk through all the files, like we did with the bills, and save out the only the important parts.


In [22]:
votes = []
for f in os.walk(os.getcwd() + "\\input\\"):
    if "vote" in f[0]:
        for file in f[-1]:
            if ".json" in file:
                fileName = f[0] + "\\" + file
                with open(fileName, "r") as b:
                    votes.append(json.loads(b.read()))


What information does each vote contain?


In [23]:
votes[0]["roll_call"]


{'roll_call_id': 903331,
 'bill_id': 1299612,
 'date': '2020-01-16',
 'desc': 'House State, Veterans, & Military Affairs: Refer House Bill 20-1036 to the Committee of the Whole.',
 'yea': 8,
 'nay': 0,
 'nv': 0,
 'absent': 1,
 'total': 9,
 'passed': 1,
 'chamber': 'H',
 'chamber_id': 21,
 'votes': [{'people_id': 14269, 'vote_id': 1, 'vote_text': 'Yea'},
  {'people_id': 16573, 'vote_id': 1, 'vote_text': 'Yea'},
  {'people_id': 18712, 'vote_id': 1, 'vote_text': 'Yea'},
  {'people_id': 18714, 'vote_id': 1, 'vote_text': 'Yea'},
  {'people_id': 20024, 'vote_id': 1, 'vote_text': 'Yea'},
  {'people_id': 20027, 'vote_id': 1, 'vote_text': 'Yea'},
  {'people_id': 20137, 'vote_id': 1, 'vote_text': 'Yea'},
  {'people_id': 20148, 'vote_id': 4, 'vote_text': 'Absent'},
  {'people_id': 20159, 'vote_id': 1, 'vote_text': 'Yea'}]}

Unsurprisingly, there's a fair amount of stuff we don't care about here. Let's clean it up and grab only the important parts. We'll do the same thing we did with bill information above.


In [24]:
vote_list = []
for vote in votes:
    vote_id = vote["roll_call"]["roll_call_id"]
    bill_vote_id = vote["roll_call"]["bill_id"]
    vote_desc = vote["roll_call"]["desc"]
    vote_passed = bool(vote["roll_call"]["passed"])
    actual_votes = []
    for v in vote["roll_call"]["votes"]:
        actual_votes.append((v["people_id"], v["vote_text"]))
    vote_list.append(
        {
            "ID": vote_id,
            "Bill ID": bill_vote_id,
            "Description": vote_desc,
            "Passed": vote_passed,
            "Votes": f"{actual_votes}",
        }
    )



In [25]:
vote_df = pd.DataFrame.from_records(vote_list)
vote_df.set_index("ID", inplace=True)
vote_df

,Bill ID,Description,Passed,Votes
ID,,,,
903331,1299612,"House State, Veterans, & Military Affairs: Ref...",True,"[(14269, 'Yea'), (16573, 'Yea'), (18712, 'Yea'..."
903370,1299570,"House State, Veterans, & Military Affairs: Ref...",True,"[(14269, 'Yea'), (16573, 'Yea'), (18712, 'Yea'..."
903416,1299603,"House State, Veterans, & Military Affairs: Ref...",True,"[(14269, 'Yea'), (16573, 'Yea'), (18712, 'Yea'..."
903812,1299547,House Business Affairs & Labor: Refer House Bi...,True,"[(14270, 'Yea'), (16567, 'Yea'), (16578, 'Yea'..."
903850,1299584,House Business Affairs & Labor: Refer House Bi...,True,"[(14270, 'Yea'), (16567, 'Yea'), (16578, 'Yea'..."
...,...,...,...,...
978242,1299069,House Education: Adopt amendment L.007,False,"[(10783, 'NV'), (14807, 'NV'), (15133, 'NV'), ..."
978243,1299069,"House Education: Refer Senate Bill 20-029, as ...",True,"[(10783, 'Yea'), (14807, 'Yea'), (15133, 'Yea'..."
978262,1298981,"House Finance: Refer Senate Bill 20-035, as am...",True,"[(14270, 'Yea'), (18710, 'Yea'), (18721, 'Yea'..."


Amazing, now let's round it out by doing the same thing to the people data.

In [12]:
people = []
for f in os.walk(os.getcwd() + "\\input\\"):
    if "people" in f[0]:
        for file in f[-1]:
            if ".json" in file:
                fileName = f[0] + "\\" + file
                with open(fileName, "r") as b:
                    people.append(json.loads(b.read()))

In [13]:
people[0]['person']

{'people_id': 10773,
 'person_hash': '3no4th7a',
 'party_id': '2',
 'state_id': 6,
 'party': 'R',
 'role_id': 2,
 'role': 'Sen',
 'name': 'Donald Coram',
 'first_name': 'Donald',
 'middle_name': 'Lee',
 'last_name': 'Coram',
 'suffix': '',
 'nickname': 'Don',
 'district': 'SD-006',
 'ftm_eid': 12998104,
 'votesmart_id': 125548,
 'opensecrets_id': '',
 'knowwho_pid': 297722,
 'ballotpedia': 'Don_Coram',
 'committee_sponsor': 0,
 'committee_id': 0,
 'state_federal': 0}

In [14]:
people_list = []
for person in people:
    person_id = person["person"]["people_id"]
    party = person["person"]["party"]
    role = person["person"]["role"]
    name = person["person"]["name"]
    district = person["person"]["district"]
    people_list.append(
        {
            "ID": person_id,
            "Name": name,
            "Party": party,
            "Role": role,
            "District": district,
        }
    )



In [15]:
people_df = pd.DataFrame.from_records(people_list)
people_df.set_index("ID", inplace=True)
people_df

,Name,Party,Role,District
ID,,,,
10773,Donald Coram,R,Sen,SD-006
10778,Ray Scott,R,Sen,SD-007
10780,Pete Lee,D,Sen,SD-011
10781,Angela Williams,D,Sen,SD-033
10783,James Wilson,R,Rep,HD-060
...,...,...,...,...
3301,Lois Court,D,Sen,SD-031
3312,Nancy Todd,D,Sen,SD-028
3323,Kevin Priola,R,Sen,SD-025


Brilliant, now let's close it out by saving these to 3 csv files for later use. This may seem superfluous with only one session's worth of data, but this will definitely come in handy later when pulling all available data. (Depending on size issues, this may even need to be made into a pickle file at some point. For now we will keep it as csv to make it human-readable)

In [22]:
with open('./input/CO/2020-2020_Regular_Session/bills.csv', 'w') as f:
    bill_df.to_csv(f)
with open('./input/CO/2020-2020_Regular_Session/people.csv', 'w') as f:
    people_df.to_csv(f)
with open('./input/CO/2020-2020_Regular_Session/votes.csv', 'w') as f:
    vote_df.to_csv(f)